In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('option-chain-equity-derivatives.csv')
df.head()

CALLS  \
NaN OI CHNG IN OI VOLUME IV LTP      CHNG      BID QTY BID PRICE ASK PRICE ASK QTY STRIKE PRICE BID QTY BID PRICE ASK PRICE ASK QTY CHNG  LTP  IV     VOLUME  CHNG IN OI   
    2  -          -      -  3,390.75 -3,390.75 -       -         5,450.75  1,125   6,100.00     1,275   0.55      0.60      2,100   -0.15 0.55 159.69 1,555          581   
    -  -          -      -  -        -         -       -         5,400.80  1,125   6,150.00     6,075   0.10      1.50      150     -     -    -      -                -   
                                                                 5,350.85  1,125   6,200.00     3,000   0.20      1.50      150     -     -    -      -                -   
                                                                 5,300.85  1,125   6,250.00     1,050   0.10      1.50      150     -     -    -      -                -   

                                                                                                                                                             Unnamed: 1  \
NaN OI CHNG IN OI VOLUME IV LTP      CHNG      BID QTY BID PRICE ASK PRICE ASK QTY STRIKE PRICE BID QTY BID PRICE ASK PRICE ASK QTY CHNG  LTP  IV     VOLUME         OI   
    2  -          -      -  3,390.75 -3,390.75 -       -         5,450.75  1,125   6,100.00     1,275   0.55      0.60      2,100   -0.15 0.55 159.69 1,555         802   
    -  -          -      -  -        -         -       -         5,400.80  1,125   6,150.00     6,075   0.10      1.50      150     -     -    -      -               -   
                                                                 5,350.85  1,125   6,200.00     3,000   0.20      1.50      150     -     -    -      -               -   
                                                                 5,300.85  1,125   6,250.00     1,050   0.10      1.50      150     -     -    -      -               -   

                                                                                                                                                              PUTS  
NaN OI CHNG IN OI VOLUME IV LTP      CHNG      BID QTY BID PRICE ASK PRICE ASK QTY STRIKE PRICE BID QTY BID PRICE ASK PRICE ASK QTY CHNG  LTP  IV     VOLUME   NaN  
    2  -          -      -  3,390.75 -3,390.75 -       -         5,450.75  1,125   6,100.00     1,275   0.55      0.60      2,100   -0.15 0.55 159.69 1,555    NaN  
    -  -          -      -  -        -         -       -         5,400.80  1,125   6,150.00     6,075   0.10      1.50      150     -     -    -      -        NaN  
                                                                 5,350.85  1,125   6,200.00     3,000   0.20      1.50      150     -     -    -      -        NaN  
                                                                 5,300.85  1,125   6,250.00     1,050   0.10      1.50      150     -     -    -      -        NaN

In [9]:
calls_df = df['Unnamed: 1']

calls_df.to_csv('Calls')

data_df = pd.read_csv('Calls')

data_df.head()

removing header and making first row as new header

In [13]:
data_df.columns = data_df.iloc[0]
data_df = data_df[1:]

data_df.to_csv('Calls_refined')

resetting the index and data cleaning

In [14]:
refined_df = pd.read_csv('Calls_refined')

refined_df.reset_index(drop = True, inplace = True)

refined_df.drop(['Unnamed: 0','Unnamed: 1'], axis = 1, inplace = True)

In [15]:
refined_df.head()

,OI,CHNG IN OI,VOLUME,IV,LTP,CHNG,BID QTY,BID PRICE,ASK PRICE,ASK QTY,STRIKE PRICE,BID QTY.1,BID PRICE.1,ASK PRICE.1,ASK QTY.1,CHNG.1,LTP.1,IV.1,VOLUME.1,OI.1
0,2,-,-,-,"3,390.75","-3,390.75",-,-,"5,450.75","1,125","6,100.00","1,275",0.55,0.60,"2,100",-0.15,0.55,159.69,"1,555",802
1,-,-,-,-,-,-,-,-,"5,400.80","1,125","6,150.00","6,075",0.10,1.50,150,-,-,-,-,-
2,-,-,-,-,-,-,-,-,"5,350.85","1,125","6,200.00","3,000",0.20,1.50,150,-,-,-,-,-
3,-,-,-,-,-,-,-,-,"5,300.85","1,125","6,250.00","1,050",0.10,1.50,150,-,-,-,-,-
4,-,-,-,-,-,-,-,-,"5,250.90","1,125","6,300.00","1,050",0.10,1.50,150,-,-,-,-,-


replacing the - with zeros for calculation purpose

In [16]:
refined_df.replace('-', 0, inplace = True)

replacing commas in numbers and making numbers float data type from earlier string

In [17]:
refined_df.replace(',','', regex=True, inplace=True)

c = refined_df.select_dtypes(object).columns
refined_df[c] = refined_df[c].apply(pd.to_numeric,errors='ignore')


calculating the put loss

In [18]:
refined_df['PUT Loss'] = 0

for i in range(len(refined_df['STRIKE PRICE'])):
    loss = 0
    for j in range(len(refined_df['STRIKE PRICE'])):
        if refined_df['STRIKE PRICE'][j]-refined_df['STRIKE PRICE'][i] >0:
            loss = loss + (refined_df['STRIKE PRICE'][j]-refined_df['STRIKE PRICE'][i])*refined_df['OI.1'][j]
#         print(loss)
            refined_df['PUT Loss'][i] =loss 

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


Calculating the call loss

In [19]:
refined_df['CALL Loss'] = 0
for i in range(len(refined_df['STRIKE PRICE'])):
    loss = 0
    for j in range(len(refined_df['STRIKE PRICE'])):
        if refined_df['STRIKE PRICE'][j]-refined_df['STRIKE PRICE'][i] <0:
            loss = loss + (refined_df['STRIKE PRICE'][j]-refined_df['STRIKE PRICE'][i])*refined_df['OI'][j]*-1
#         print(loss)
            refined_df['CALL Loss'][i] =loss 

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


total loss calculation

In [20]:
refined_df['Total Loss'] = refined_df['CALL Loss'] +refined_df['PUT Loss']

In [21]:
refined_df.head()

,OI,CHNG IN OI,VOLUME,IV,LTP,CHNG,BID QTY,BID PRICE,ASK PRICE,ASK QTY,...,ASK PRICE.1,ASK QTY.1,CHNG.1,LTP.1,IV.1,VOLUME.1,OI.1,PUT Loss,CALL Loss,Total Loss
0,2,0,0,0.0,3390.75,-3390.75,0,0.0,5450.75,1125,...,0.6,2100,-0.15,0.55,159.69,1555,802,380737650,0,380737650
1,0,0,0,0.0,0.00,0.00,0,0.0,5400.80,1125,...,1.5,150,0.00,0.00,0.00,0,0,373807700,100,373807800
2,0,0,0,0.0,0.00,0.00,0,0.0,5350.85,1125,...,1.5,150,0.00,0.00,0.00,0,0,366877750,200,366877950
3,0,0,0,0.0,0.00,0.00,0,0.0,5300.85,1125,...,1.5,150,0.00,0.00,0.00,0,0,359947800,300,359948100
4,0,0,0,0.0,0.00,0.00,0,0.0,5250.90,1125,...,1.5,150,0.00,0.00,0.00,0,0,353017850,400,353018250


In [22]:
refined_df["STRIKE PRICE"][refined_df['Total Loss'].idxmin()]

9400.0

In [23]:
refined_df.to_csv("Max Pain.csv")

refined_df['STRIKE PRICE'][refined_df['Total Loss'].nsmallest(5).index]